<a href="https://colab.research.google.com/github/cedamusk/final-year/blob/main/basecode22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install obspy matplotlib tensorflow

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from obspy import UTCDateTime, Stream, Trace
from obspy.signal.trigger import classic_sta_lta
from obspy.signal.filter import bandpass
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
def generate_synthetic_data(num_samples, sample_rate, event_duration, noise_level):
  time=np.arange(num_samples)/sample_rate
  background=np.random.normal(0, noise_level, num_samples)

  event_start=num_samples//4
  event_end=event_start+int(event_duration*sample_rate)
  event=np.zeros(num_samples)
  event[event_start:event_end]=np.sin(2*np.pi*5*(time[event_start:event_end]-time[event_start]))*np.exp(-(time[event_start:event_end]-time[event_start])/0.2)

  data=background+event
  return data, event, event_start, event_end

In [ ]:
num_samples=10000
sample_rate=100
event_duration=2
noise_level=0.1

synthetic_data, true_event, event_start, event_end=generate_synthetic_data(num_samples, sample_rate, event_duration, noise_level)

In [ ]:
start_time=UTCDateTime("2021-01-01T00:00:00")
trace=Trace(data=synthetic_data)
trace.stats.starttime=start_time
trace.stats.delta=1.0/sample_rate
trace.stats.channel='SHZ'
stream=Stream([trace])

In [ ]:
filtered_stream=stream.copy()
filtered_stream.filter('bandpass', freqmin=0.5, freqmax=20, corners=4, zerophase=True)

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(np.arange(num_samples)/sample_rate, synthetic_data)
plt.title("Synthetic Seismic Data")
plt.xlabel('Time(s)')
plt.ylabel("Amplitude")
plt.show()

In [ ]:
filtered_stream=stream.copy()
filtered_stream.filter('bandpass', freqmin=0.5, freqmax=20, corners=4, zerophase=True)

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(np.arange(num_samples)/sample_rate, filtered_stream[0].data)
plt.title("Filtered Seismic Data")
plt.xlabel("Time(s)")
plt.ylabel("Amplitude")
plt.show()

In [ ]:
def sta_lta_detection(stream, sta_length, lta_length, threshold):
  data=stream[0].data
  sta_lta=classic_sta_lta(data, sta_length, lta_length)
  triggers=np.where(sta_lta>threshold)[0]
  return triggers, sta_lta

In [ ]:
sta_length=int(0.5*sample_rate)
lta_length=int(5*sample_rate)
threshold=1
triggers, sta_lta=sta_lta_detection(filtered_stream, sta_length, lta_length, threshold)

In [ ]:
plt.figure(figsize=(12, 8))
plt.subplot(2,1,1)
plt.plot(np.arange(num_samples)/sample_rate, filtered_stream[0].data)
plt.title("Filtered Seismic Data")
plt.xlabel("Time(s)")
plt.ylabel("Amplitude")
for trigger in triggers:
  plt.axvline(x=trigger/sample_rate, color='r', linestyle='--')

plt.subplot(2,1,2)
plt.plot(np.arange(len(sta_lta))/sample_rate, sta_lta)
plt.axhline(y=threshold, color='r', linestyle='--')
plt.title("STA/LTA Ratio")
plt.tight_layout()
plt.show()

In [ ]:
window_size=100
step=10


In [ ]:
def create_windows(data, window_size, step):
  return np.array([data[i:i+window_size]for i in range(0, len(data)-window_size+1, step)])

In [ ]:
X=create_windows(filtered_stream[0].data, window_size, step)
y=np.zeros(len(X))

In [ ]:
for trigger in triggers:
  event_windows=np.where((np.arange(len(X))*step<=trigger)&(trigger<np.arange(len(X))*step+window_size))[0]


In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train_mean=X_train.mean()
X_train_std=X_train.std()
X_train=(X_train-X_train_mean)/X_train_std
X_test=(X_test-X_train_mean)/X_train_std

In [ ]:
X_train=X_train.reshape((X_train.shape[0], X_train.shape[1],1))
X_test=X_test.reshape((X_test.shape[0], X_test.shape[1],1))

In [ ]:
model=Sequential([
    SimpleRNN(64, input_shape=(window_size, 1), activation='relu', return_sequences=True),
    SimpleRNN(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history=model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"test accuracy:{accuracy}")


In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
def rnn_detection(data, model, window_size, step):
  X=create_windows(data, window_size, step)
  X=(X-X_train_mean)/X_train_std
  X=X.reshape((X.shape[0], X.shape[1],1))
  predictions=model.predict(X)
  return predictions.flatten()

In [ ]:
rnn_predictions=rnn_detection(filtered_stream[0].data, model, window_size, step)

In [ ]:
plt.figure(figsize=(12, 8))
plt.subplot(3,1,1)
plt.plot(np.arange(num_samples)/sample_rate, filtered_stream[0].data)
plt.title("Filtered Seismic data")
plt.xlabel("Time(s)")
plt.ylabel("Amplitude")

plt.subplot(3,1,2)
plt.plot(np.arange(len(sta_lta))/sample_rate, sta_lta)
plt.axhline(y=threshold, color='r', linestyle='--')
plt.title("STA/LTA Ratio")
plt.xlabel('Time(s)')
plt.ylabel("STA/LTA Ratio")

plt.subplot(3,1,3)
plt.plot(np.arange(len(rnn_predictions))*step/sample_rate, rnn_predictions)
plt.title("RNN Predictions")
plt.xlabel("Time(s)")
plt.ylabel("Probability")

plt.tight_layout()
plt.show()


In [ ]:
num_rnn_predictions=len(rnn_predictions)
sta_lta_trimmed=sta_lta[:num_rnn_predictions]
combined_detections=(sta_lta_trimmed>threshold)&(rnn_predictions>0.5)
combined_triggers=np.where(combined_detections)[0]

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(np.arange(num_samples)/sample_rate, filtered_stream[0].data)
plt.title("Filtered Seismic data with Combined detections")
plt.xlabel("Time(s)")
plt.ylabel("Amplitude")
for trigger in combined_triggers:
  plt.axvline(x=trigger/sample_rate, color='g', linestyle='--')
  plt.show()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
true_events=np.zeros(len(filtered_stream[0].data))
true_events[event_start:event_end]=1

In [ ]:
sta_lta_predictions=np.zeros(len(filtered_stream[0].data))
sta_lta_predictions[triggers]=1

In [ ]:
rnn_predictions_full=np.zeros(len(filtered_stream[0].data))
rnn_predictions_full[np.arange(len(rnn_predictions))*step]=(rnn_predictions>0.5).astype(int)

In [ ]:
combined_predictions=np.zeros(len(filtered_stream[0].data))
combined_predictions[combined_triggers]=1

In [ ]:
def print_metrics(y_true, y_pred, method_name):
  precision=precision_score(y_true, y_pred)
  recall=recall_score(y_true, y_pred)
  f1=f1_score(y_true, y_pred)
  print(f"{method_name}Metrics:")
  print(f"Precision:{precision:.4f}")
  print(f"Recall:{recall:.4f}")
  print(f'f1 Score:{f1:.4f}')
  print()

In [ ]:
print_metrics(true_events, sta_lta_predictions, "STA/LTA")
print_metrics(true_events, rnn_predictions_full, "RNN")
print_metrics(true_events, combined_predictions, "Combined STA/LTA and RNN")